# Task:
 - ## Create a GUI app by tkinter.
 - ## Two functionalities:
     - ### 1st: Scan; search for the input entered by scanning all directories or desired one and return all file names similar to the input.
     - ### 2nd: Merge; copy the data from all the files of desired extension and dump into one single file having the same extension.
     
### Note: there's a catch in my logicality i.e. in this GUI if you want to search for any specific extension, you'd have to enter it properly--what I mean to say is if you enter '.doc', you won't get any results for '.docx' and the same goes for vice-versa. 

In [1]:
from tkinter import *
from tkinter import messagebox
import os, logging as lg, docx, PyPDF2

In [2]:
class Log:
    def __init__(self):
        try:
            self.logFile="SearchandMerge.log"
            if os.path.exists(self.logFile):
                os.remove(self.logFile)
            lg.basicConfig(filename=self.logFile, level=lg.INFO, format="%(asctime)s %(levelname)s %(message)s")
            #Adding the StreamHandler to record logs in the console.
            self.console_log = lg.StreamHandler()
            self.console_log.setLevel(lg.INFO) #setting level to the console log.
            self.format = lg.Formatter("%(asctime)s %(message)s")
            self.console_log.setFormatter(self.format) #defining format for the console log.
            lg.getLogger('').addHandler(self.console_log) #adding handler to the console log.
        
        except Exception as e:
            lg.info(e)
            
        else:
            lg.info("Log Class successfully executed!")

In [3]:
class GUI:
    def __init__(self):
        """
        Whole layout of our desired wibdow is set here as wwe know __init__ function will be 
        automatically called the moment we create its object.
        """
        self.win = Tk()
        self.win.title("Search & Merge")
        
        #directories
        self.drive = ""
        self.drives = self.__mydrives()
        
        #a dict that stores filenames and their roots as well.
        self.fileRoot = {}
        #current dir where merged files will be created
        self.curDir = os.getcwd()
        
        try:
            #row0
            self.prompt = "Enter the desired drive from (C, D, E or G) \nOR\n Enter 'scan all' or 'search all' if you so much want so: "
            self.l = Label(self.win, text = self.prompt)
            self.l.grid(row=0, column=0)

            #row1
            self.searchInput = Entry(self.win, borderwidth=3, width=125) #search input
            self.searchInput.grid(row=1, column=0)
            self.chooseDrive = Button(self.win, text="select Drive", padx=8, pady=3, command=self.__choosedrive).grid(row=1, column=1) #fro choosing dir

            #row2
            self.l2 = Label(self.win)
            self.l2.grid(row = 2, column = 0) #what directory has been chosen.

            #row3
            self.searchFor = Button(self.win, text="search", padx=10, pady=5, command=self.__search_for)
            self.searchFor.grid(row = 3, column = 0)

            #row4
            self.l4 = Label(self.win)
            self.l4.grid(row = 4, column = 0)

            #row5
            self.searchRes = Text(self.win, borderwidth=2, width=110) #search results
            self.searchRes.grid(row=5, column=0, columnspan=4)
            self.searchRes.config(state="disabled") #so that nothing couldn't be typed into it externally.
            #adding a vertical scroll bar.
            self.vsbar = Scrollbar(self.win, orient='vertical')
            self.vsbar.grid(row = 5, column = 3, sticky = 'e')
            self.vsbar.config(command=self.searchRes.yview)

            #row6
            self.merger = Button(self.win, text="Merge (.txt, .docx or .pdf)", padx=8, pady=3, command=self.__merge).grid(row = 6, column = 0)
        except Exception as e:
            lg.error(e)
        finally:
            self.win.mainloop()
            
    def __mydrives(self):
        """
        Will return the list of available drives on this pc.
        """
        try:
            import string
            alphas = string.ascii_uppercase #generating all alphabets.
            dri = []
            dri+=alphas
            mydrives = [] #will contain available drives of this system.
            for i in dri:
                if os.path.exists(i+"://"):
                    mydrives.append(i)
            return mydrives
        except Exception as e:
            lg.error(e)            
        
        
    def __choosedrive(self):
        """
        Will assist in choosing a directory.
        """
        try:
            self.l2.config(text = "") #truncating this label everytime this button is triggered.
            self.drive = self.searchInput.get().capitalize()

            if self.drive in self.drives:
                self.l2.config(text = f"Your desired drive {self.drive} will be scanned!\nNow, enter search input..")
                self.searchInput.delete(0, END)
                return self.drive

            elif self.drive == "Search all" or self.drive == "Scan all":
                self.l2.config(text = "All drives will be scanned!\nNow, enter search input..")
                self.searchInput.delete(0, END)
                return self.drive
            
            elif len(self.drive)==0:
                messagebox.showwarning("No drive selected!", "for the least part, try choosing a drive..")
                lg.error("No drive selected!")
            else:
                messagebox.showerror("No such drive found!", "Enter an appropriate drive, my man!")
                lg.error("An appropriate drive needs to be chosen!")
                self.searchInput.delete(0, END)
        except Exception as e:
            lg.error(e)
    
    def __search_for(self): #functionality for search button.
        try:
            self.fileRoot = {} #truncating the dict every time search button is hit.
            s = self.searchInput.get() #searchable i.e. we'll search for it.
            self.searchInput.delete(0, END)
            
            if self.l2["text"] == "": #i.e. no drive selected and search button is hit.
                messagebox.showwarning("no drive selected!", "for the least part, try choosing a drive..")
                lg.error("No drive selected!")
                return 0
                
            if len(s) == 0: #if nothing is entered.
                messagebox.showwarning("no search input!", "to get some results and in order to merge them, something oughta be entered!")
                lg.error("to get some results and in order to merge them, something oughta be entered!")
                return 0 #to not further advance in this function.
            
            #Clearing the prior search results if any before searching again.
            self.searchRes.config(state="normal")
            self.searchRes.delete(1.0, END) 
            self.searchRes.config(state="disabled")
            
            self.l4.config(text = f"search results for '{s}'..")

            if self.drive == "Scan all" or self.drive == "Search all": #If all directories are to be searched.
                
                lg.info("All drives are being scanned..")
                lg.info("generating search results...")
                
                disks = [i+"://" for i in self.drives] #a list containing drives with their paths.
                for d in disks:
                    for root, dirs, files in os.walk(d, topdown=True): 
                        for i in files:
                            if s.startswith("."): #search input is an extension
                                if i.endswith(s):
                                    self.searchRes.config(state="normal")
                                    self.searchRes.insert(END, i)
                                    self.searchRes.insert(END, '\n')
                                    self.searchRes.config(state="disabled")
                                    
                                    self.fileRoot[i] = root #adding filename and its root as well to fileRoot{}.
                                    lg.info(i)
                            else:
                                if i.startswith(s) or s in i: 
                                    """
                                    if filenames start with search input or search input is present anywhere in between in filenames.
                                    """
                                    self.searchRes.config(state="normal")
                                    self.searchRes.insert(END, i)
                                    self.searchRes.insert(END, '\n')
                                    self.searchRes.config(state="disabled")
                                    
                                    self.fileRoot[i] = root #adding filename and its root as well to fileRoot{}.
                                    lg.info(i)
                if len(self.searchRes.get("1.0", END))==1:
                    messagebox.showinfo("nothing found!", "try entering some different input to be searched, No results found!")
                    lg.info("No results found!")

            else: #if single directory is to be searched.
                
                lg.info(f"Drive {self.drive} is being scanned..")
                lg.info("generating search results...")
                
                for root, dirs, files in os.walk(f'{self.drive}://', topdown=True):
                    for i in files:
                        if s.startswith("."): #search input is an extension
                            if i.endswith(s):
                                
                                self.searchRes.config(state="normal")
                                self.searchRes.insert(END, i)
                                self.searchRes.insert(END, '\n')
                                self.searchRes.config(state="disabled")
                                
                                self.fileRoot[i] = root #adding filename and its root as well to fileRoot{}.
                                lg.info(i)
                        else:
                            if i.startswith(s) or s in i: 
                                """
                                if filenames start with search input or search input is present anywhere in between in filenames.
                                """
                                
                                self.searchRes.config(state="normal")
                                self.searchRes.insert(END, i)
                                self.searchRes.insert(END, '\n')
                                self.searchRes.config(state="disabled")
                                
                                self.fileRoot[i] = root #adding filename and its root as well to fileRoot{}.
                                lg.info(i)
                if len(self.searchRes.get("1.0", END))==1:
                    messagebox.showinfo("nothing found!", "try entering some different input to be searched, No results found!")
                    lg.info("No results found!")

        except Exception as e:
            lg.error(e)     
        else:
            lg.info("search results generated!")
            lg.info("-------------------------")
            
    def __merge(self):
        global content
        content = [i for i in self.fileRoot.keys()]
        
        try:
            if ".txt" in content[0]:

                txt_file="txt(Merged).txt"
                if os.path.exists(txt_file):
                    os.remove(txt_file)

                for i in content: #parsing each .txt file contained in the 'content'.
                    if i==txt_file: #skipping the merged file that is meant to be truncated everytime.
                        continue
                    os.chdir(self.fileRoot[i]) #going to the dir where the file is.

                    with open(i, "r+", encoding='utf-8') as f:
                        data = f.readlines()
                        f.close()
                        
                    os.chdir(self.curDir) #coming back to our original directory.
                    with open(txt_file, "a+") as g:
                        g.writelines(i)
                        g.write("\n")
                        g.write("\n")
                        g.writelines(data)
                        g.write("\n")
                        g.write("\n")
                        
                g.close()
                messagebox.showinfo("file ready!", "Merged '.txt' is ready! You can check it now.")
                lg.info(f"Merged '.txt' is ready! You can check it now.")

            elif ".docx" in content[0]:
                docx_file = "docx(Merged).docx"
                if os.path.exists(docx_file):
                    os.remove(docx_file)

                merged = docx.Document() #creating our merged .docx file
                for i in content:
                    if i==docx_file: #to skip the file we are creating, if it has already been lying there.
                        continue
                    os.chdir(self.fileRoot[i]) #going to the dir where the file to be copied is. 
                    doc = docx.Document(i)
                    os.chdir(self.curDir) #returning to our original dir.
                    merged.add_heading(i)
                    merged.add_paragraph("\n")
                    for j in range(len(doc.paragraphs)):
                        merged.add_paragraph(doc.paragraphs[j].text)
                    merged.add_paragraph("\n") 
                merged.save(docx_file)
                messagebox.showinfo("file ready!", "Merged '.docx' is ready! You can check it now.")
                lg.info(f"Merged '.docx' is ready! You can check it now.")

            elif ".pdf" in content[0]:
                pdf_file = "pdf(Merged).pdf"
                if os.path.exists(pdf_file): #deleting the merged pdf if it already exists.
                    os.remove(pdf_file)
                    
                merged = PyPDF2.PdfMerger() #creating a merger object.
                for i in content:
                    if i==pdf_file:
                        continue
                    os.chdir(self.fileRoot[i])
                    #if the file is encrypted
                    reader = PyPDF2.PdfReader(i)
                    if reader.is_encrypted:
                        continue
                    else:    
                        merged.append(i) #appending .pdf to the merger.
                os.chdir(self.curDir) #getting back to our orignal dir.
                merged.write(pdf_file)
                merged.close()
                messagebox.showinfo("file ready!", "Merged '.pdf' is ready! You can check it now.")
                lg.info("Merged '.pdf' is ready! You can check it now.")
                
            else:
                messagebox.showinfo("non-mergeable extension!", "files of the entered extension couldn't be merged..")
                lg.info("files of the entered extension coudn't be merged..")

        except Exception as e:
            lg.error(e) 

In [4]:
Log()
guiApp = GUI()
guiApp

2022-06-11 20:00:47,470 Log Class successfully executed!
2022-06-11 20:01:00,632 Drive C is being scanned..
2022-06-11 20:01:00,632 generating search results...
2022-06-11 20:01:00,643 hwcompat.txt
2022-06-11 20:01:00,645 hwcompatPE.txt
2022-06-11 20:01:00,648 hwexclude.txt
2022-06-11 20:01:00,651 hwexcludePE.txt
2022-06-11 20:01:00,653 idwbinfo.txt
2022-06-11 20:01:00,681 erofflps.txt
2022-06-11 20:01:00,731 oobe1.txt
2022-06-11 20:01:00,733 sub1.txt
2022-06-11 20:01:00,736 oobe4.txt
2022-06-11 20:01:00,738 sub4.txt
2022-06-11 20:01:00,765 NetflixChannelIdHP.txt
2022-06-11 20:01:00,767 CustomDataHP.txt
2022-06-11 20:01:00,770 PASS1_ML2.txt
2022-06-11 20:01:00,781 dummy.txt
2022-06-11 20:01:00,798 vollist.txt
2022-06-11 20:01:00,858 CASSANDRA-14092.txt
2022-06-11 20:01:00,859 CHANGES.txt
2022-06-11 20:01:00,860 LICENSE.txt
2022-06-11 20:01:00,861 NEWS.txt
2022-06-11 20:01:00,862 NOTICE.txt
2022-06-11 20:01:00,865 README.txt
2022-06-11 20:01:00,867 README.txt
2022-06-11 20:01:00,877 me-